# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 28 2022 8:39AM,253881,12,HH-39022,Hush Hush,Released
1,Dec 28 2022 8:39AM,253883,19,8959131,"CLINUVEL, Inc.",Released
2,Dec 28 2022 8:39AM,253882,12,HH8650,Hush Hush,Released
3,Dec 28 2022 8:30AM,253866,10,SOIVA0000570,Ivaoes Animal Health,Released
4,Dec 28 2022 8:30AM,253872,10,0086270803,ISDIN Corporation,Released
5,Dec 28 2022 8:30AM,253872,10,0086270804,ISDIN Corporation,Released
6,Dec 28 2022 8:30AM,253865,10,0086270811,ISDIN Corporation,Released
7,Dec 28 2022 8:30AM,253865,10,0086270810,ISDIN Corporation,Released
8,Dec 28 2022 8:30AM,253865,10,0086270806,ISDIN Corporation,Released
9,Dec 28 2022 8:30AM,253865,10,0086270807,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,253878,Released,1
41,253879,Released,23
42,253881,Released,1
43,253882,Released,1
44,253883,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253878,NaN,NaN,1.0
253879,NaN,NaN,23.0
253881,NaN,NaN,1.0
253882,NaN,NaN,1.0
253883,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253790,3.0,0.0,2.0
253795,0.0,0.0,61.0
253798,0.0,0.0,40.0
253800,0.0,0.0,64.0
253804,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253790,3,0,2
253795,0,0,61
253798,0,0,40
253800,0,0,64
253804,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253790,3,0,2
1,253795,0,0,61
2,253798,0,0,40
3,253800,0,0,64
4,253804,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253790,3,,2
1,253795,,,61
2,253798,,,40
3,253800,,,64
4,253804,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 28 2022 8:39AM,253881,12,Hush Hush
1,Dec 28 2022 8:39AM,253883,19,"CLINUVEL, Inc."
2,Dec 28 2022 8:39AM,253882,12,Hush Hush
3,Dec 28 2022 8:30AM,253866,10,Ivaoes Animal Health
4,Dec 28 2022 8:30AM,253872,10,ISDIN Corporation
6,Dec 28 2022 8:30AM,253865,10,ISDIN Corporation
51,Dec 28 2022 8:30AM,253863,10,ISDIN Corporation
64,Dec 28 2022 8:30AM,253870,10,ISDIN Corporation
69,Dec 28 2022 8:30AM,253868,10,ISDIN Corporation
81,Dec 28 2022 8:30AM,253862,10,"Virtue Rx, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 28 2022 8:39AM,253881,12,Hush Hush,,,1
1,Dec 28 2022 8:39AM,253883,19,"CLINUVEL, Inc.",,,1
2,Dec 28 2022 8:39AM,253882,12,Hush Hush,,,1
3,Dec 28 2022 8:30AM,253866,10,Ivaoes Animal Health,,,1
4,Dec 28 2022 8:30AM,253872,10,ISDIN Corporation,,,28
5,Dec 28 2022 8:30AM,253865,10,ISDIN Corporation,,,38
6,Dec 28 2022 8:30AM,253863,10,ISDIN Corporation,,,1
7,Dec 28 2022 8:30AM,253870,10,ISDIN Corporation,,,7
8,Dec 28 2022 8:30AM,253868,10,ISDIN Corporation,,,3
9,Dec 28 2022 8:30AM,253862,10,"Virtue Rx, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 28 2022 8:39AM,253881,12,Hush Hush,1,,
1,Dec 28 2022 8:39AM,253883,19,"CLINUVEL, Inc.",1,,
2,Dec 28 2022 8:39AM,253882,12,Hush Hush,1,,
3,Dec 28 2022 8:30AM,253866,10,Ivaoes Animal Health,1,,
4,Dec 28 2022 8:30AM,253872,10,ISDIN Corporation,28,,
5,Dec 28 2022 8:30AM,253865,10,ISDIN Corporation,38,,
6,Dec 28 2022 8:30AM,253863,10,ISDIN Corporation,1,,
7,Dec 28 2022 8:30AM,253870,10,ISDIN Corporation,7,,
8,Dec 28 2022 8:30AM,253868,10,ISDIN Corporation,3,,
9,Dec 28 2022 8:30AM,253862,10,"Virtue Rx, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 28 2022 8:39AM,253881,12,Hush Hush,1,,
1,Dec 28 2022 8:39AM,253883,19,"CLINUVEL, Inc.",1,,
2,Dec 28 2022 8:39AM,253882,12,Hush Hush,1,,
3,Dec 28 2022 8:30AM,253866,10,Ivaoes Animal Health,1,,
4,Dec 28 2022 8:30AM,253872,10,ISDIN Corporation,28,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 28 2022 8:39AM,253881,12,Hush Hush,1.0,NaN,NaN
1,Dec 28 2022 8:39AM,253883,19,"CLINUVEL, Inc.",1.0,NaN,NaN
2,Dec 28 2022 8:39AM,253882,12,Hush Hush,1.0,NaN,NaN
3,Dec 28 2022 8:30AM,253866,10,Ivaoes Animal Health,1.0,NaN,NaN
4,Dec 28 2022 8:30AM,253872,10,ISDIN Corporation,28.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 28 2022 8:39AM,253881,12,Hush Hush,1.0,0.0,0.0
1,Dec 28 2022 8:39AM,253883,19,"CLINUVEL, Inc.",1.0,0.0,0.0
2,Dec 28 2022 8:39AM,253882,12,Hush Hush,1.0,0.0,0.0
3,Dec 28 2022 8:30AM,253866,10,Ivaoes Animal Health,1.0,0.0,0.0
4,Dec 28 2022 8:30AM,253872,10,ISDIN Corporation,28.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6599641,548.0,1.0,3.0
12,507763,2.0,0.0,0.0
16,253806,1.0,0.0,0.0
19,2538571,45.0,10.0,4.0
22,761590,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6599641,548.0,1.0,3.0
1,12,507763,2.0,0.0,0.0
2,16,253806,1.0,0.0,0.0
3,19,2538571,45.0,10.0,4.0
4,22,761590,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,548.0,1.0,3.0
1,12,2.0,0.0,0.0
2,16,1.0,0.0,0.0
3,19,45.0,10.0,4.0
4,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,548.0
1,12,Released,2.0
2,16,Released,1.0
3,19,Released,45.0
4,22,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,22
Status,,,,,
Completed,3.0,0.0,0.0,4.0,0.0
Executing,1.0,0.0,0.0,10.0,0.0
Released,548.0,2.0,1.0,45.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,22
0,Completed,3.0,0.0,0.0,4.0,0.0
1,Executing,1.0,0.0,0.0,10.0,0.0
2,Released,548.0,2.0,1.0,45.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,22
0,Completed,3.0,0.0,0.0,4.0,0.0
1,Executing,1.0,0.0,0.0,10.0,0.0
2,Released,548.0,2.0,1.0,45.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()